Note: Deep Learning with PyTorch: A 60 Minute Blitz
====
---
<https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html>

Lesson3: Neural Networks
----
**Question1: optimizier**

[1] torch.nn   
[2] torch.nn.Module   
[3] torch.autograd   

**3.1 preview**

[1] Neural networks can be contructed using the `torch.nn` package.   
[2] `nn` depends on `autograd` to define models and differentiate them.   
[3] An `nn.Module` contains layers, and a method `forward(input)` that returns the `output`.   
[4] A typical training procedure for neural network is as follows:   
- Define the nerual network that has some learnable parameters (or weights)   
- Iterate over a dataset of inputs   
- Process input through the network   
- Compute the loss (how far is the output from being correct)  
- Propagate gradients back into the network's parameters  
- Update the weights of the network, typically using a simple update rule: `weight = weight - learning_rate * gradient`  

**3.2 Define the network**

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    
    def __init__(self):
        super(Net,self).__init__()
        # 1 input image channel,
        # 6 output channels
        # 3*3 square convolution kernel
        self.conv1 = nn.Conv2d(1,6,3)
        self.conv2 = nn.Conv2d(6,16,3)
        # an affine operation: y = Wx +b
        self.fc1 = nn.Linear(16*6*6, 120) # 6*6 from image dimension
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
    def forward(self,x):
        # Max pooling over a (2,2) window
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self,x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)    

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [17]:
# The learnable parameters of a model are returned by net.parameters()
params = list(net.parameters())
print(len(params)) # 5 layers, each with a set of weights and a bias.
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


In [20]:
# Let's try a random 32*32 input
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[-0.1472,  0.0489,  0.0650, -0.1106, -0.0622,  0.1114,  0.0175,  0.0435,
         -0.0825,  0.0254]], grad_fn=<AddmmBackward>)


In [21]:
# Zero the gradient buffers of all parameters and backprops with random 
# gradients:
net.zero_grad()
out.backward(torch.randn(1,10))

**Recap:**   
[1] `torch.Tensor` - A multi-dimensional array with support for autograd
operations like `backward()`. Also holds the gradient w.r.t the tensor.  
[2] `nn.Module` - Neural network module. Convenient way of encapsulating parameters, with helpers for moving them to GPU, exporting, loading, etc. 
[3] `nn.Parameter` - A kind of Tensor, that is automatically registered as a parameter when assigned as an attribute to a `Module`.    

**3.3 Loss Function**

In [33]:
output = net(input)
target = torch.randn(10) # a dummy target, for example
target = target.view(1,-1) # make it the same shape as output
criterion = nn.MSELoss()
loss = criterion(output,target)
print(loss)

tensor(0.4348, grad_fn=<MseLossBackward>)


In [25]:
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # RuLU

**3.4 Backprop**

In [34]:
net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)
loss.backward()
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0061, -0.0029,  0.0051, -0.0026, -0.0063,  0.0013])


**3.5 Update the weights**

The simplest update rule used in practice is the Stochastic Gradient Descent(SGD)  
`weight = weight - learning_rate * gradient`

In [35]:
# We can implement this using simple Python code:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [36]:
print(net.conv1.bias.grad)

tensor([ 0.0061, -0.0029,  0.0051, -0.0026, -0.0063,  0.0013])


In [43]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01,momentum=0.9)
print(net.conv1.bias.grad)
# in your training loop:
optimizer.zero_grad() # zero the gradient buffers
print(net.conv1.bias.grad)
output = net(input)
loss = criterion(output, target)
loss.backward()
print(net.conv1.bias.grad)
optimizer.step()
print(net.conv1.bias.grad)
# Question here, why the last two have the same bias,grad. because I use optimizier, 
# I think I should have different grad. 

tensor([0.2441, 0.2265, 0.2418, 0.1205, 0.1897, 0.2212])
tensor([0., 0., 0., 0., 0., 0.])
tensor([-0.0037, -0.0041, -0.0051, -0.0033,  0.0030, -0.0058])
tensor([-0.0037, -0.0041, -0.0051, -0.0033,  0.0030, -0.0058])
